# About: ホスト環境の設定ファイル変更
---

ホスト環境の `/etc/rsyslog.conf` などの設定ファイルを編集します。

## 概要

構築環境のホスト側の `/etc/` などに配置されている設定ファイルを編集する手順を示します。

![処理の流れ](images/moodle-040-01.png)

主な手順は以下のようになります。

1. 構築環境の設定ファイルをローカルのJupyter Notebook環境に取得する
2. 取得したファイルのバックアップを作成する
3. Notebookの編集機能を利用してローカル環境に取得したファイルの編集をおこなう
4. 編集後のファイルを構築環境に配置する

## パラメータの指定

### グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を指定します。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

#### チェック

指定された `target_group` の値が適切なものかチェックします。

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    target_file = Path('group_vars') / f"{target_group}.yml"
    if target_file.is_file():
        target_dir.mkdir(exist_ok=True)
        target_file.rename(target_dir / "00-moodle.yml")
    else:
        raise RuntimeError(f"ERROR: not exists {target_group}")

`target_group`で指定したホストにAnsibleで到達可能であることを確認します。

In [ ]:
!ansible {target_group} -m ping

### 編集対象の設定ファイルを指定する

編集対象となるファイルのパスを指定します。

In [ ]:
# (例)
# target_file = '/etc/rsyslog.conf'

target_file = 

## 設定ファイルの編集

設定ファイルをローカル環境に取得して、Jupyter Notebookの編集機能を用いて設定ファイルを編集します。

![設定ファイルの取得、編集](images/moodle-040-02.png)

次のセルを実行すると、以下の手順が実行されます。

1. 構築環境の設定ファイルをローカル環境に取得する
2. 取得した設定ファイルのバックアップを作成する
3. Jupyter Notebookの編集機能を利用して設定ファイルを編集するためのリンクを表示する

In [ ]:
%run scripts/edit_conf.py
fetch_host_conf(target_group, target_file)

上のセルの出力に表示されているリンクをクリックして設定ファイルの編集を行ってください。

 メニューの[Edit]-[Find]で検索文字を入力することで、編集箇所に移動できます。
> また、ファイルの編集後は**必ず**、メニューの[File]-[Save]を選択してファイルの保存を行ってください。

ローカル環境に取得したファイルは、以下のパスに格納されています。

`./edit/{target_group}/{YYYYMMDDHHmmssffffff}/{設定ファイルのファイル名}`

`{target_group}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

また、バックアップファイルは以下のパスに格納されます。

`./edit/{target_group}/{YYYYMMDDHHmmssffffff}/{設定ファイルのファイル名}.orig`

次のセルを実行すると編集の前後での差分を確認することができます。

In [ ]:
show_local_host_conf_diff(target_group, target_file)

## 編集した設定ファイルの配置

編集したファイルを構築環境に配置します。

![設定ファイルの配置](images/moodle-040-03.png)

次のセルを実行すると、以下の手順が実行されます。

1. 編集前と編集後の差分を表示する
2. 編集した設定ファイルを構築環境に配置する

In [ ]:
upload_host_conf(target_group, target_file)

## 変更を取り消す

編集前のファイルを構築環境に配置して、もとの設定に戻します。

![設定ファイル変更の取り消し](images/moodle-040-04.png)

次のセルを実行すると、以下の手順が実行されます。

1. 編集後と編集前のファイルの差分を表示する
2. 編集前のファイルを構築環境に配置する

In [ ]:
revert_host_conf(target_group, target_file)